In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Aug  3 22:55:26 2017

@author: sudhanshu
"""

import pandas as pd 
import gzip 
def parse(path): 
    g = gzip.open(path, 'rb') 
    for l in g: 
        yield eval(l) 

def getDF(path): 
    i = 0 
    df = {} 
    for d in parse(path): 
         df[i] = d
         i += 1 
    return pd.DataFrame.from_dict(df, orient='index') 

df = getDF("C:\\Users\\sudhanshu\\Desktop\\Springboard\\Capstone 1\\reviews_Automotive_5.json.gz")

#Checking Shape and descriptive characteristics of the corpus
print(df.shape)
print(df.describe())



(20473, 9)
            overall  unixReviewTime
count  20473.000000    2.047300e+04
mean       4.471841    1.365018e+09
std        0.940337    3.621266e+07
min        1.000000    1.121386e+09
25%        4.000000    1.354838e+09
50%        5.000000    1.373414e+09
75%        5.000000    1.390435e+09
max        5.000000    1.405901e+09


In [2]:
import matplotlib.pyplot as plt
import gensim
import numpy as np
import spacy

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

import os, re, operator, warnings, codecs

from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore



from spacy.en import English
nlp = spacy.load("en")

#Calculating Length of dataframe
DfLength=len(df.index)
print(DfLength)

C:\Users\sudhanshu\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


20473


In [3]:
"""
Loop over every review and break it into sentences.
For each sentence, lematize the words, remove punctuations and save to 
a text file on the system
"""

directory = "C:\\Users\\sudhanshu\\Desktop\\Springboard\\Capstone 1\\"
FileName="Comp_Review_Sentences.txt"

filepath=directory+FileName


def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space



In [4]:
#Opening textfile
with codecs.open(filepath, 'w', encoding='utf_8') as f:
    #Iterating over each review
    for i in range(DfLength):
        #Providing each review to Spacy
        doc=nlp(df.at[i,"reviewText"])
        #Breaking review into sentences and lemmatizing each word
        for sentences in doc.sents:
            #print(u" ".join([token.lemma_ for token in sentences if not punct_space(token)])) 
            string = u" ".join([token.lemma_ for token in sentences if not punct_space(token)])
            string = re.sub('[^a-zA-Z0-9\n\.]', ' ', string)
            string = " ".join(string.split())
            f.write(string + "\n")



In [5]:
##Using Gensim to read in the file using LineSentence

unigram_sentences = LineSentence(filepath)

#Forming a Bi-Gram model: New York becomes New_York

bigram_model = Phrases(unigram_sentences)

#Saving the Bi-Gram Model
bigram_model_filepath = os.path.join(directory, 'bigram_model_all')

bigram_model.save(bigram_model_filepath)

# load the finished Bi-Gram model from disk
bigram_model = Phrases.load(bigram_model_filepath)

#Saving the results of the Bi-Gram model on disk
bigram_sentences_filepath = os.path.join(directory, "bigram_sentences_all.txt")


#Writing Bi-Gram proccessed data to file
with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')
            

#Using Gensim to read in the Bi-Gram sentence file using LineSentence

bigram_sentences = LineSentence(bigram_sentences_filepath)



            
#Forming a Tri-Gram model

Trigram_model = Phrases(bigram_sentences)

#Saving the Tri-Gram Model
Trigram_model_filepath = os.path.join(directory, 'Trigram_model_all')

Trigram_model.save(Trigram_model_filepath)

# load the finished Tri-Gram model from disk
Trigram_model = Phrases.load(Trigram_model_filepath)

#Saving the results of the Tri-Gram model on disk
Trigram_sentences_filepath = os.path.join(directory, "Trigram_sentences_all.txt")


#Writing Tri-Gram proccessed data to file and removing stopwords

"""
with codecs.open(Trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            
            Trigram_sentence = u' '.join(Trigram_model[bigram_sentence])
            
            f.write(Trigram_sentence + '\n')
"""
   
with codecs.open(Trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for bigram_sentence in bigram_sentences:
            trigram_review = Trigram_model[bigram_sentence]
            trigram_review = [term for term in trigram_review if term not in spacy.en.STOPWORDS]
            Trigram_sentence = u' '.join(trigram_review)
            
            f.write(Trigram_sentence + '\n')

#Reading in the Tri-Gram sentences to learn its dictionary 

Trigram_dictionary_filepath = os.path.join(directory,"trigram_dict_all.dict")    

Trigram_reviews = LineSentence(Trigram_sentences_filepath)       
            
#Learning the dictionary by iterating over all reviews

Trigram_dictionary = Dictionary(Trigram_reviews)


#Removing tokens that are very rare or too common from
#The dictionary (filter_extremes) and reassign integer ids (compactify)
#Remove all words that are in less than 10 sentences or in ore than 40% of the sentences 
Trigram_dictionary.filter_extremes(no_below=10, no_above=0.40)
Trigram_dictionary.compactify()

#Writing dictionary output to a file
Trigram_dictionary.save(Trigram_dictionary_filepath)

#Forming a Bag of Words implementation of the corpus
#Setting filepath to save the bag of words model
Trigram_bow_filepath = os.path.join(directory, 'trigram_bow_corpus_all.mm')

#Loading Dictionary from the disk
Trigram_dictionary = Dictionary.load(Trigram_dictionary_filepath)

#Forming a bag of words model
MmCorpus.serialize(Trigram_bow_filepath, (Trigram_dictionary.doc2bow(reviews) for reviews in LineSentence(Trigram_sentences_filepath)))          
          
#Loading the bag of words corpus from the disk
Trigram_bow_corpus = MmCorpus(Trigram_bow_filepath)  

#Forming LDA models

#Saving path name of the file
lda_model_filepath = os.path.join(directory, 'lda_model_all')


#Starting LDA modelling
with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(Trigram_bow_corpus,
                           num_topics=20,
                           id2word=Trigram_dictionary,
                           workers=3)
#Saving the model to disk    
lda.save(lda_model_filepath)


#Function to explore topic


def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print (u'{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print (u'{:20} {:.3f}'.format(term, round(frequency, 3)))        
        

C:\Users\sudhanshu\Anaconda3\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [8]:
LDAvis_prepared=pyLDAvis.gensim.prepare(lda, Trigram_bow_corpus, Trigram_dictionary)       
pyLDAvis.display(LDAvis_prepared)
